In [1]:
import matplotlib, os
import numpy as np
import imageio.v3 as imageio
from IPython.display import clear_output
from shared.Render import collect_saved_frames, render_function_point
from shared.TestFunctions import TEST_FUNCTIONS
matplotlib.use('Agg')
np.random.seed(42)

#### The particle swarm optimization algorithm

In [2]:
def particle_swarm_optimization(func, rnge, pop_size=30, iterations=50, w=0.7, c1=1.5, c2=1.5):
    v_max = 0.2 * (rnge[1] - rnge[0])
    positions = np.random.uniform(rnge[0], rnge[1], size=(pop_size, 2))
    velocities = np.random.uniform(-v_max, v_max, size=(pop_size, 2))
    fitness = np.apply_along_axis(lambda pos: func(tuple(pos)), 1, positions)
    pbest_positions = positions.copy()
    pbest_values = fitness.copy()
    gbest_index = np.argmin(fitness)
    gbest_position = positions[gbest_index].copy()
    gbest_value = fitness[gbest_index]
    yield 0, tuple(gbest_position)
    for iteration in range(1, iterations + 1):
        r1 = np.random.uniform(size=(pop_size, 2))
        r2 = np.random.uniform(size=(pop_size, 2))
        velocities = (
            w * velocities
            + c1 * r1 * (pbest_positions - positions)
            + c2 * r2 * (gbest_position - positions)
        )
        velocities = np.clip(velocities, -v_max, v_max)
        positions = positions + velocities
        positions = np.clip(positions, rnge[0], rnge[1])
        fitness = np.apply_along_axis(lambda pos: func(tuple(pos)), 1, positions)
        improved = fitness < pbest_values
        pbest_positions[improved] = positions[improved]
        pbest_values[improved] = fitness[improved]
        best_particle = np.argmin(pbest_values)
        if pbest_values[best_particle] < gbest_value:
            gbest_value = pbest_values[best_particle]
            gbest_position = pbest_positions[best_particle].copy()
            yield iteration, tuple(gbest_position)

#### Run the particle swarm optimization algorithm and collect frames for multiple functions

In [3]:
gif_folder_path = "gifs/particle_swarm"
os.makedirs(gif_folder_path, exist_ok=True)

In [4]:
for func, params in TEST_FUNCTIONS.items():
    print(f"Test function: {func.__name__}")
    rnge = (params[0], params[1])
    resolution = params[2]
    for idx, point in particle_swarm_optimization(func, rnge):
        frame = render_function_point(func, rnge=rnge, resolution=resolution, point=point, iteration=idx)
        imageio.imwrite(os.path.join(gif_folder_path, f"{func.__name__}{idx:08d}.png"), frame)
    collect_saved_frames(gif_folder_path, f"{str.capitalize(func.__name__)}.gif", fps=5)
clear_output()

#### Visualize the particle swarm optimization algorithm on multiple functions

![Sphere](gifs/particle_swarm/Sphere.gif)
![Ackley](gifs/particle_swarm/Ackley.gif)
![Rastrigin](gifs/particle_swarm/Rastrigin.gif)
![Rosenbrock](gifs/particle_swarm/Rosenbrock.gif)
![Griewank](gifs/particle_swarm/Griewank.gif)
![Schwefel](gifs/particle_swarm/Schwefel.gif)
![Levy](gifs/particle_swarm/Levy.gif)
![Michalewicz](gifs/particle_swarm/Michalewicz.gif)
![Zakharov](gifs/particle_swarm/Zakharov.gif)